## This simple notebook shows how to compute the validity, satisfiability, and entailment for logical sentences and to set up a simple KB agent for propositional logic with functions from AIMA logic.py

You can run this on you local computer if your current directory is your hw4 directory since all of the files it needs are included in it.

To run on Colab, make sure you are logged into Google, uncomment the lines in the following cell, and run it once. This asks the operating system to pip install a package required by logic.py, clone the aima repository in your temporarary file system and then cd to the aima directory so you can import python files.

Yui may find it helpgul to look at the code in the aima [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) file


In [1]:
#!pip -q install ipythonblocks
#!git clone https://github.com/aimacode/aima-python.git;
#%cd aima-python

In [2]:
from logic import *

The AIMA logic.py function **expr** to create an **Expression object** from a string in which identifiers are automatically defined as Symbols and ==> is treated as an infix |'==>'|, as are <== and <=>. If the argument is already an Expression, it is returned unchanged.

In [3]:
foo = expr('P ==> Q')

The expr class (in utils.py) has a __repr__ method that is called when an instance is printed to return a string representation.

In [4]:
print(f"type:{type(foo)}; {foo}")

type:<class 'utils.Expr'>; (P ==> Q)


Since a **knowledge base** (KB) is just a set of sentences that are all taken to be true, we can represent it as a conjunction of sentences.  This next expr can be thought of as a KB with three sentences.

In [5]:
expr('P & (P ==> Q) & (~P ==> R)')

((P & (P ==> Q)) & (~P ==> R))

The **tt_true** function checks an expression object to see if it is valid, i.e., true in all possible models.  A valid sentence is true for all possible assignments of true and false to its variables.

In [6]:
tt_true(expr('P'))

False

In [7]:
tt_true(expr('P | ~ P'))

True

In [8]:
tt_true(expr('(P ==> Q) <=> (~P | Q)'))

True

**dpll_satisfiable** checks satisfiability of a propositional sentence, returning a *model* if it is satisfiable and *False* if not.  The model is represented as dictionary with the propositional variables as the keys and True or Fase as their values.

In [9]:
dpll_satisfiable(expr('P & Q'))

{P: True, Q: True}

Note that if there are multiple ways that a sentence can be satisfied, only one model is shown and it may be a minimal model, i.e., not including vaues for variables that can be either True or False.

In [10]:
dpll_satisfiable(expr('P | Q'))

{P: True}

In [11]:
dpll_satisfiable(expr('P & ~Q'))

{P: True, Q: False}

For this example, if Q is true, P can be either True or False, so this is a minimal model.

In [12]:
dpll_satisfiable(expr('P ==> Q'))

{P: False}

dpll_satisfiable takes a single sentence, but it can be arbitrarily comlex, like this conjunction of three simple expressions

In [13]:
dpll_satisfiable(expr('P & (P ==> Q) & (~P ==> R)'))

{R: True, Q: True, P: True}

The following KB can not be satisfied, since if P is true, then Q must also be true, but the KB says that Q is false.

In [14]:
dpll_satisfiable(expr('P & (P ==> Q) & (~P ==> R) & ~Q'))

False

**tt_entails** uses a simple model checking proceedure can determine if a knowledge base (a conjunction of propositional expressions) entails another (see section 7.4 in our text)

In [15]:
tt_entails(expr('P & Q'), expr('Q'))

True

In [16]:
tt_entails(expr('P | Q'), expr('Q'))

False

We can also use **WalkSAT**, a function that uses local search.  It takes a list of clauses and checks for their satisfiability by randomly flipping values of the variables in them.  It returns None if no solution can be found within its alloted flips, which has a default of 10k.  This can often solve larger problems than dpll, but, although it is **sound**, it is not **complete**.

In [17]:
WalkSAT([expr('P & (P ==> Q) & (~P ==> R)')])

{P: True, Q: True, R: False}

WalkSAT returns None if it cannot find a model that satisfies the list of expressions.  Of course, Python does not print anything for input that evaluates to None, as the following eample shows.

In [18]:
WalkSAT([expr('P & (P ==> Q) & (~P ==> R) & ~Q')])

Here's a case there the set of sentences is empty, so it shiuld return an empty model in the form of an empty dictionary

In [19]:
WalkSAT([])

{}

**PropKB** is  a class for a KB of propositions.  We'll create **kb1** as a new knowledge base

In [20]:
kb1 = PropKB()

and add several sentences to it

In [21]:
kb1.tell(expr('P ==> Q'))
kb1.tell(expr('Q ==> R'))

The KB does not know and cannot infer that R is True

In [22]:
kb1.ask_if_true(expr('R'))

False

If we add P to the KB, it can infer Q and then R

In [23]:
kb1.tell(expr('P'))

In [24]:
kb1.ask_if_true(expr('R'))

True

The clauses property of a KB holds the facts in conjunctive normal form

In [25]:
kb1.clauses

[(Q | ~P), (R | ~Q), P]

If we have an inconsistant KB, this simple proceedure wil prove that anything is true

In [26]:
kb1.tell(expr('~P'))
kb1.clauses

[(Q | ~P), (R | ~Q), P, ~P]

In [27]:
kb1.ask_if_true(expr('X'))

True

In [28]:
kb1.ask_if_true(expr('~X'))

True

### *fin*